# ExaModelsPower.jl Documentation

## Overview
ExaModelsPower.jl can model large-scale optimal power flow (OPF) problems using the ExaModels package to generate models that can be solved using either CPU or GPU. This tutorial will demonstrate how ExaModelsPower.jl can be leveraged to solve different versions of the OPF, and how the user can customize the solving technique to better match their needs. Currently, all models generated by ExaModelsPower represent the full, AC version of the OPF formulation without any simplifications. 


## Installation
The latest version of ExaModelsPower can be installed in julia as so. Additionally, in order to develop models that can be solved on the GPU, CUDA is required. 

In [ ]:
using ExaModelsPower, CUDA


[info | PowerModels]: Suppressing information and warning messages for the rest of this session.  Use the Memento package for more fine-grained control of logging.


In order to solve the ExaModels developed by ExaModelsPower, an NLP solver is required. ExaModels is compatible with MadNLP and Ipopt, but this tutorial will focus on MadNLP to demonstrate GPU solving capabilities

In [ ]:
using MadNLP, MadNLPGPU #, NLPModelsIpopt 

Finally, we install ExaModels to allow solved models to be unpacked

In [ ]:
using ExaModels

## Static OPF
We will begin by constructing and solving a static OPF using the function opf_model. For the static OPF, the only input required is the filename for the OPF matpower file. The file does not need to be locally installed, and it will be automatically downloaded from __[power-grid-library](https://github.com/power-grid-lib/pglib-opf)__ if the file is not found in the user's data folder. If keywords are not specified, the numerical type will default to Float64, the backend will default to nothing (used on CPU) and the form will default to polar coordinates. 

In [ ]:
model, vars, cons = opf_model(
    "pglib_opf_case118_ieee.m";
    backend = CUDABackend(),
    form = :polar,
    T = Float64
)

[ Info: Loading cached JLD2 file


Once the model is built, we can generate a solution using MadNLP

In [5]:
result = madnlp(model; tol=1e-6)

This is MadNLP version v0.8.5, running with cuDSS v0.4.0

Number of nonzeros in constraint Jacobian............:     5925
Number of nonzeros in Lagrangian Hessian.............:     8474

Total number of variables............................:     1088
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      970
                     variables with only upper bounds:        0
Total number of equality constraints.................:      981
Total number of inequality constraints...............:      558
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      186
        inequality constraints with only upper bounds:      372

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.7806508e+02 2.76e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  4.8050149e+02 2.76e+00 1.01e+02  -1.0 3.40e+00    -  3.02e-03 1.63e-

"Execution stats: Optimal Solution Found (tol = 1.0e-06)."

Once a solution has been generated, the values of any of the variables in the model can be unpacked using the vars NamedTuple

In [16]:
solution(result, vars.vm)

118-element CuArray{Float64, 1, CUDA.DeviceMemory}:
 1.0580725147176768
 1.0115214146045532
 1.0371945440885082
 1.012711563921921
 1.0170524545909747
 1.054436931704562
 1.0575146839021525
 1.0303511034466952
 1.0418501518770769
 1.0519359527784118
 1.0262877364063878
 1.0520047889645077
 0.9891027940049588
 ⋮
 1.0419729787069323
 1.0508468790943595
 1.0302731418574345
 1.0245508038382196
 0.9931136288424807
 1.0451418653107944
 1.0195019536760748
 1.007491293024812
 1.0573020624029843
 1.0514577985618567
 1.0467048629513434
 1.0331889703347816

Result also stores the objective value

In [18]:
result.objective

97210.50257421451

ExaModelsPower supports solving the OPF in either polar or rectangular coordinates

In [19]:
model, vars, cons = opf_model(
    "pglib_opf_case118_ieee.m";
    form = :rect
)
result = madnlp(model; tol=1e-6)
result.objective

[ Info: Loading cached JLD2 file


This is MadNLP version v0.8.5, running with umfpack

Number of nonzeros in constraint Jacobian............:     6770
Number of nonzeros in Lagrangian Hessian.............:     8577

Total number of variables............................:     1053
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      817
                     variables with only upper bounds:        0
Total number of equality constraints.................:      981
Total number of inequality constraints...............:      676
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      304
        inequality constraints with only upper bounds:      372

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.7806508e+02 2.76e+00 2.19e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  3.4420227e+04 2.00e+00 7.98e+01  -1.0 2.86e+00    -  3.58e-03 2.74e-01h  

97213.56501338647

In this case, the objective value and performance speed is comparable. However, for some cases, MadNLP can only solve the problem on one of the two available coordinate systems

## Multi Period OPF
The multi-period (MP) OPF can also be modeled in ExaModelsPower with a number of user-specified adjustments, including storage. 

Some of the models in this portion of the tutorial involve using external files. While we provide the necessary code to access these additional files, you may view them __[here](https://github.com/mit-shin-group/multi-period-opf-data)__ as well

We will start with the simplest way to model the MPOPF, which also does not require the user to have any data already downloaded. Instead, the user specifies a demand curve for the system. The demand curve is a vector of ratios from 0 to 1 which indicate the scaling of demand compared to the demand indicated by the static OPF file. In this model of the MPOPF, every consuming bus has the same scaling in power demand for each point in time. A corrective action ratio, which limits the ramp rate of generators, can also be inputted. It is set to 0.1 as a default. The adjustable coordinate system and backend that were present for the static OPF are also available for all MPOPF models

In [20]:
model, vars, cons = mpopf_model(
    "pglib_opf_case118_ieee.m", # static network data
    [.64, .60, .58, .56, .56, .58, .64, .76, .87, .95, .99, 1.0, .99, 1.0, 1.0,
    .97, .96, .96, .93, .92, .92, .93, .87, .72, .64], #Demand curve
    backend = CUDABackend(),
    corrective_action_ratio = 0.3
)
result = madnlp(model; tol=1e-6)

[ Info: Loading cached JLD2 file


This is MadNLP version v0.8.5, running with cuDSS v0.4.0

Number of nonzeros in constraint Jacobian............:   150717
Number of nonzeros in Lagrangian Hessian.............:   211850

Total number of variables............................:    27200
                     variables with only lower bounds:        0
                variables with lower and upper bounds:    24250
                     variables with only upper bounds:        0
Total number of equality constraints.................:    25365
Total number of inequality constraints...............:    14406
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:     5106
        inequality constraints with only upper bounds:     9300

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.1951627e+04 2.76e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  1.2003121e+04 2.76e+00 1.01e+02  -1.0 3.70e+00    -  3.02e-03 1.63e-

"Execution stats: Optimal Solution Found (tol = 1.0e-06)."

If the user would like to provide more complex demand profiles, they can provide their own input files. The number of rows in each input file should match the number of buses in the static OPF datafile, and the number of columns will dictate the number of time periods in the MP model

First, download the example load profile datafiles

In [ ]:
using Downloads

# Define URLs for Pd and Qd data files (raw or raw GitHub links)
pd_url = "https://raw.githubusercontent.com/mit-shin-group/multi-period-opf-data/refs/heads/main/halfhour_30.Pd"
qd_url = "https://raw.githubusercontent.com/mit-shin-group/multi-period-opf-data/refs/heads/main/halfhour_30.Qd"

# Define local paths to temporarily store the files
pd_file = "halfhour_30.Pd"
qd_file = "halfhour_30.Qd"

# Download the files if they don't already exist
if !isfile(pd_file)
    Downloads.download(pd_url, pd_file)
end

if !isfile(qd_file)
    Downloads.download(qd_url, qd_file);
end




"halfhour_30.Qd"

Next, build the MPOPF model, providing the dynamic load data instead of a demand curve as input

In [26]:
# Run your model
model, vars, cons = mpopf_model(
    "pglib_opf_case30_ieee.m",  # static network data (assumed local or already handled)
    pd_file,                    # dynamic load data (Pd)
    qd_file;                    # dynamic load data (Qd)
    backend = CUDABackend()
)

# Solve
result = madnlp(model; tol=1e-6)


[ Info: Downloading pglib_opf_case30_ieee.m
[ Info: Loading MATPOWER file
[ Info: Saving JLD2 cache file


This is MadNLP version v0.8.5, running with cuDSS v0.4.0

Number of nonzeros in constraint Jacobian............:    39438
Number of nonzeros in Lagrangian Hessian.............:    56100

Total number of variables............................:     7080
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     6180
                     variables with only upper bounds:        0
Total number of equality constraints.................:     6866
Total number of inequality constraints...............:     3748
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:     1288
        inequality constraints with only upper bounds:     2460

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.9926674e+03 3.06e-01 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  2.0068381e+03 3.06e-01 1.01e+02  -1.0 2.23e+00    -  4.64e-03 4.00e-

"Execution stats: Optimal Solution Found (tol = 1.0e-06)."

### MPOPF with storage
The MPOPF model can also be constructed with storage considerations. We model storage using the model proposed in __[Geth, Coffrin, Fobes 2020](https://arxiv.org/pdf/2004.14768)__. This requires inputting a modified datafile containing storage parameters. When modeling MPOPF with storage, all of the aforementioned tuneable parameters are still available. We also allow the user to specify whether or not to model the charging/discharging complementarity constraint. This is set to false by default to avoid potential numerical error.

First, we download the modified datafile with storage parameters included

In [ ]:
# Define URL for the main datafile
stor_url = "https://raw.githubusercontent.com/mit-shin-group/multi-period-opf-data/refs/heads/main/pglib_opf_case30_ieee_mod.m"

# Define local path to temporarily store the file
stor_file = "pglib_opf_case30_ieee_mod.m"

# Download the file if it doesn't already exist
if !isfile(stor_file)
    Downloads.download(stor_url, stor_file);
end


"pglib_opf_case30_ieee_mod.m"

Generate the model with your modified datafile. If the datafile contains storage parameters, ExaModelsPower will automatically recognize it and include the additional necessary constraints. 

In [ ]:
model, vars, cons = mpopf_model(
    stor_file, # static network data with storage parameters
    pd_file,                    # dynamic load data (Pd)
    qd_file;                    # dynamic load data (Qd)
    backend = CUDABackend(),
    storage_complementarity_constraint = false
)
result = madnlp(model; tol=1e-6)
result.objective

[ Info: Loading MATPOWER file
[ Info: Saving JLD2 cache file


This is MadNLP version v0.8.5, running with cuDSS v0.4.0

Number of nonzeros in constraint Jacobian............:    40696
Number of nonzeros in Lagrangian Hessian.............:    56580

Total number of variables............................:     7500
                     variables with only lower bounds:       60
                variables with lower and upper bounds:     6420
                     variables with only upper bounds:        0
Total number of equality constraints.................:     7106
Total number of inequality constraints...............:     3868
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:     1348
        inequality constraints with only upper bounds:     2520

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.0037435e+03 3.06e-01 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  2.0174516e+03 3.06e-01 1.01e+02  -1.0 2.01e+00    -  5.16e-03 4.00e-

299068.04048929544

ExaModelsPower also provides a secondary option to avoid dealing with complementarity constraints. The user can specify a function that computesloss in battery level as a smooth function of discharge rate and the storage devices thermal rating parameter. We provide an arbitrary example function to demonstrate the modeling capability

In [29]:
function example_function(d, srating)
    return d + .2/srating*d^2
end

example_function (generic function with 1 method)

ExaModelsPower will automatically adjust the necessary constraints if one of the inputs provided is a function

In [30]:
model, vars, cons = mpopf_model(
    stor_file, # static network data with storage parameters
    pd_file,                    # dynamic load data (Pd)
    qd_file,                    # dynamic load data (Qd)
    example_function;           # discharge function
    backend = CUDABackend(),
    storage_complementarity_constraint = false
)
result = madnlp(model; tol=1e-6)
result.objective

[ Info: Loading cached JLD2 file


This is MadNLP version v0.8.5, running with cuDSS v0.4.0

Number of nonzeros in constraint Jacobian............:    40516
Number of nonzeros in Lagrangian Hessian.............:    56580

Total number of variables............................:     7500
                     variables with only lower bounds:       60
                variables with lower and upper bounds:     6420
                     variables with only upper bounds:        0
Total number of equality constraints.................:     7106
Total number of inequality constraints...............:     3868
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:     1348
        inequality constraints with only upper bounds:     2520

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.0037435e+03 3.06e-01 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  2.0180947e+03 3.06e-01 1.01e+02  -1.0 2.10e+00    -  4.93e-03 4.00e-

299044.3240453915

Despite the example function being generated somewhat arbitrarily, the resultant objective values remain quite close for both the smooth and piecewise charge/discharge functions.